In [1]:
import numpy as np 
import pandas as pd # Data Processing 
from sklearn.model_selection import train_test_split # Categorize trainset, validation set 

import torch
import torch.nn as nn 
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset 

import glob # File to list
from tqdm import tqdm # Process bar 

from sklearn.preprocessing import MinMaxScaler



In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(ResidualBlock, self).__init__()
        self.hidden1 = nn.Linear(hidden_dim, hidden_dim) 
        self.leaky_relu1 = nn.LeakyReLU(negative_slope=0.01)

        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.glu = nn.GLU() # Gated Linear Unit splits feature into half >> ex 128 to 64.

        self.hidden3 = nn.Linear(hidden_dim//2, hidden_dim)
        self.leaky_relu3 = nn.LeakyReLU(negative_slope=0.01)

        self.hidden4 = nn.Linear(hidden_dim, hidden_dim) 
        self.glu2 = nn.GLU() # half

        self.hidden5 = nn.Linear(hidden_dim//2, input_dim) 
        self.leaky_relu5 = nn.LeakyReLU(negative_slope=0.01)

        # 32 x 64 / 128 x 64

    def forward(self, x):
        out = self.leaky_relu1(self.hidden1(x))
        out = self.glu(self.hidden2(out))
        out = self.leaky_relu3(self.hidden3(out))
        out = self.glu2(self.hidden4(out))
        out = self.leaky_relu5(self.hidden5(out))
        return out

class MovePredictionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MovePredictionModel, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        #self.leaky_relu = nn.LeakyReLU(negative_slope=0.3)

        # Residual block
        self.residual_block = ResidualBlock(input_dim, hidden_dim)

        # self.hidden_skiplayer = nn.Linear(hidden_dim//2, input_dim)
        self.hidden_skiplayer = nn.Linear(input_dim, hidden_dim)
        self.leakyrelu_skiplayer = nn.LeakyReLU(negative_slope=0.01)

        #self.hidden = nn.Linear(hidden_dim, output_dim)
        #self.leakyrelu = nn.LeakyReLU(negative_slope=0.02)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        #processed_input = self.leaky_relu(self.input_layer(x))
        processed_input = self.input_layer(x)
        residual_out = self.residual_block(processed_input) # output tensor size: input_dim
        
        out = x + residual_out
        out = self.hidden_skiplayer(out)
        out = self.leakyrelu_skiplayer(out)
        out = self.output_layer(out)
        return out



In [3]:
INPUT_DIM = 9 # Linear vel + Linear acc + Angular vel + Angular acc + pos diff(vel*delta t) + rot diff(angvel * delta t) + bIsDrifting
HIDDEN_DIM = 128
OUTPUT_DIM = 3 # Pos diff x y  and rot diff z 
EPOCHS = 5000
BATCH_SIZE = 32 # TODO
LEARNING_RATE = 0.0005

OUTPUT_FILENAME = 'model/test.pth'
DATA_PATH = 'data'

cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [11]:
# Read .csv files with pandas
file_path = "data/data0.csv"
data = pd.read_csv(file_path) # data frame 

# csv_file_paths = glob.glob(f'{DATA_PATH}/**/*.csv', recursive=True)
# for path in csv_file_paths:
#     print(path)

# Convert to tensor 
x_features_csv = [i for i in range(9)] # 0 - 8 idx
y_features_csv = [i for i in range(9, 12)] # 9 - 11 idx 

input_features = data.values[:,x_features_csv] # 0 - 8
output_features = data.values[:, y_features_csv]

# Min max scaling
x_min, x_max = input_features.min(axis=0), input_features.max(axis=0)
y_min, y_max = output_features.min(axis=0), output_features.max(axis=0)

input_features_scaled = (input_features - x_min) / (x_max - x_min)
output_features_scaled = (output_features - y_min) / (y_max - y_min)

X = torch.tensor(input_features_scaled, dtype=torch.float32)
y = torch.tensor(output_features_scaled, dtype=torch.float32)

print(X)
#print(X.shape, y.shape)

# Dataset split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
 
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# print(f'Training samples: {len(train_loader.dataset)}')
# print(f'Validation samples: {len(val_loader.dataset)}')

batch = next(iter(train_loader))  # 첫 번째 배치 가져오기
print(batch[0].shape) 


tensor([[0.5164, 0.7387, 0.5978,  ..., 0.5164, 0.5000, 0.5908],
        [0.5205, 0.5191, 0.5081,  ..., 0.5205, 0.5000, 0.6030],
        [0.5234, 0.5209, 0.5042,  ..., 0.5234, 0.4958, 0.6161],
        ...,
        [0.4160, 0.5927, 0.5218,  ..., 0.4160, 0.5007, 0.2357],
        [0.4154, 0.5180, 0.4932,  ..., 0.4154, 0.5042, 0.2297],
        [0.4032, 0.4501, 0.4555,  ..., 0.4032, 0.5042, 0.2217]])
torch.Size([32, 9])


In [12]:
# Set model
model = MovePredictionModel(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
#model = ResidualBlock(INPUT_DIM, HIDDEN_DIM)
if cuda_available:
    model.cuda()

epochs = EPOCHS
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

# TODO: Early stopping

for epoch in range(epochs):
    # Tranining 
    model.train()
    #for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} Training"):
    for X_batch, y_batch in train_loader:
        if cuda_available:
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
        
        optimizer.zero_grad()

        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if epoch % 2 == 0 or epoch == epochs-1:

        model.eval()

        val_rmse = []
        y_preds = []
        y_actuals = []
        with torch.no_grad():
            #for X_batch, y_batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} Validation"):
            for X_batch, y_batch in val_loader:

                ## if you have GPU
                if cuda_available:
                    X_batch, y_batch = X_batch.cuda(), y_batch.cuda()

                # inference the model
                y_pred = model(X_batch)

                # calculate RMSE
                rmse = torch.sqrt(criterion(y_pred, y_batch)).cpu().numpy()
                val_rmse.append(rmse)

                # for the first batch
                if len(y_preds) == 0:  
                    y_preds = y_pred.cpu().numpy()
                    y_actuals = y_batch.cpu().numpy()
                # for the rest of the batches
                else:  
                    y_preds = np.vstack((y_preds, y_pred.cpu().numpy()))
                    y_actuals = np.vstack((y_actuals, y_batch.cpu().numpy()))
        epoch_val_rmse = np.mean(val_rmse)
        print(f"Epoch {epoch+1}, Validation RMSE: {epoch_val_rmse}")

Epoch 1, Validation RMSE: nan
Epoch 3, Validation RMSE: nan
Epoch 5, Validation RMSE: nan
Epoch 7, Validation RMSE: nan
Epoch 9, Validation RMSE: nan
Epoch 11, Validation RMSE: nan
Epoch 13, Validation RMSE: nan
Epoch 15, Validation RMSE: nan
Epoch 17, Validation RMSE: nan
Epoch 19, Validation RMSE: nan
Epoch 21, Validation RMSE: nan
Epoch 23, Validation RMSE: nan
Epoch 25, Validation RMSE: nan
Epoch 27, Validation RMSE: nan
Epoch 29, Validation RMSE: nan
Epoch 31, Validation RMSE: nan
Epoch 33, Validation RMSE: nan
Epoch 35, Validation RMSE: nan
Epoch 37, Validation RMSE: nan
Epoch 39, Validation RMSE: nan
Epoch 41, Validation RMSE: nan
Epoch 43, Validation RMSE: nan
Epoch 45, Validation RMSE: nan
Epoch 47, Validation RMSE: nan


KeyboardInterrupt: 